# WhoIS Import

You only get 500 credits with https://api.ip2whois.com/v2. But you can register another email address to continue on if you need

In [228]:
import json
import time
import requests
import pandas as pd
import re
from config import *

### API constants

In [230]:
apiKey = apiKeyWHOIS
url = "https://api.ip2whois.com/v2"
querystring = {"key":apiKey,"format":"json"}

### support functions

In [216]:
def stripName(nameStr):
    pattern = "(?<=://).*"
    try:
        newName = re.findall(pattern, nameStr)[0]
    except:
        newName = nameStr
    
    if newName[-1] == '/':
        return newName[:-1]
    else:
        return newName

In [218]:
def crawlJSON(JSONdict):
    entryDict = {}
    
    level1header = list(JSONdict)
    if "error" in level1header:
        return {}
    
    for name in level1header:
        if type(JSONdict[name]) == type({}):
            level2names = list(JSONdict[name])        
        
            for name2 in level2names:
                entryDict[name + " " + name2] = JSONdict[name][name2]
        else:
            entryDict[name] = JSONdict[name]

    return entryDict

### MAIN

In [220]:
# load ScamWebsites file
scamsites = pd.read_csv('ScamWebsites.csv')
scamsites = scamsites[scamsites["API response"] != 200]
scamsites = scamsites[["Requested domain", "Domain name"]]
scamsites.shape

(527, 2)

In [221]:
for row_idx in range(scamsites.shape[0]):
    row = scamsites.iloc[row_idx]
    if row["Domain name"] != row["Domain name"]: # nan
        scamsites.iloc[row_idx]["Domain name"] = stripName(row["Requested domain"])

In [ ]:
# if you stopped halfway, try will work. If you don't have a file that already exists, it will go to except
try:
    alreadyHave = pd.read_csv("WHOIS_missing_domains.csv")

    #find idx
    lastSiteChecked = alreadyHave["Domain name"].values[-1]
    start_idx = list(scamsites["Domain name"].values).index(lastSiteChecked)
    df = alreadyHave
    domainsToCheck = scamsites["Domain name"].values[start_idx+1:]
except:
    df = pd.DataFrame()
    domainsToCheck = scamsites["Domain name"].values

### make request

In [226]:
its = 0

for name in domainsToCheck:
    querystring["domain"] = name
    response = requests.request("GET", url, params=querystring)
    time.sleep(1)
    response_dict = json.loads(response.text)
    response_dict["Domain name"] = name
    
    if "error" in response_dict:
        print(name)
        continue
    
    df = df.append(crawlJSON(response_dict), ignore_index=True)
    
    its += 1
    
    if its % 50 == 0:
        df.to_csv("WHOIS_missing_domains.csv", index=False)

giqqhwgdq.bnkas.net/i1d9.html
spp6c8jale-www.anewwayoffriday.com/#/login?code=wzojtf&tab=2
d.krakenfly.xyz/?id=60079c26897537fd1b329c405c3a4e36
down.959435.com/ijgq.html
d.gdacvipa.buzz/?id=3437c2252ebe44294b698b4d397a3205
d.app26526.xyz/?id=edc2c06fd20389553f3b2cd15cdaa178
d.krakencla.com/?id=60079c26897537fd1b329c405c3a4e36
www.elvannyou.com
select-global.com
wwwy.sdpyion.com
engxin22.com
d.app10581.xyz/?id=d16a1dd71954e8ed83ae8fb490905551
d.gdacvipa.xyz/?id=3437c2252ebe44294b698b4d397a3205
d.z2.wtf/6k1
www.telebitex.com
88678956.jsgj011.top:8967/index.php
itestfilght.com/app/1577014610
www.redeflag.com/bkopnw
spp6me7kzzs-www.anewwayoffriday.com/#/login
nl66sv1.cn
www.huosuqian5555.top/jnbluw
d.pfidcflu.buzz/?id=abe7ff16cf545fcf4c73853ca198709c
www.charsin.com/app
download.anyingouji.com
forppea.penaker.com
h5.myrtlesau.com
m.wns.gh5685.cn
www.duo-shou.cn/18500
orpbj.ygxw18.website/9wm
d.app91576.top/?id=d16a1dd71954e8ed83ae8fb490905551
trw8wh2lcz-www.5gflare.com/#/login
d.becexgbly.

In [227]:
# final save
df.to_csv("WHOIS_missing_domains.csv", index=False)